In [1]:
import json
import os
from dotenv import load_dotenv, find_dotenv
from fhir_kindling import upload_bundle, query_server, FhirServer
from fhir_kindling.serde import load_bundle, validate_bundle
from fhir.resources.patient import Patient
from fhir.resources.codeableconcept import CodeableConcept
from fhir.resources.coding import Coding
from fhir.resources.reference import Reference
from fhir.resources.observation import Observation
from fhir.resources.condition import Condition
import pandas as pd

from fhir_kindling.generators import PatientGenerator

In [2]:
%load_ext autoreload
%autoreload 2

In [25]:
load_dotenv(find_dotenv())

username = os.getenv("FHIR_USER")
password = os.getenv("FHIR_PW")
token = os.getenv("FHIR_TOKEN")

blaze_api = "http://localhost:8080/fhir"

server = FhirServer(
        api_address=blaze_api,
        client_id=os.getenv("CLIENT_ID"),
        client_secret=os.getenv("CLIENT_SECRET"),
        oidc_provider_url=os.getenv("OIDC_PROVIDER_URL")
    )


Requesting new token


In [9]:
Observation.schema()

{'title': 'Observation',
 'description': "Disclaimer: Any field name ends with ``__ext`` does't part of\nResource StructureDefinition, instead used to enable Extensibility feature\nfor FHIR Primitive Data Types.\n\nMeasurements and simple assertions.\nMeasurements and simple assertions made about a patient, device or other\nsubject.",
 'type': 'object',
 'properties': {'resource_type': {'title': 'Resource Type',
   'const': 'Observation',
   'type': 'string'},
  'fhir_comments': {'title': 'Fhir Comments',
   'element_property': False,
   'anyOf': [{'type': 'string'},
    {'type': 'array', 'items': {'type': 'string'}}]},
  'id': {'title': 'Logical id of this artifact',
   'description': 'The logical id of the resource, as used in the URL for the resource. Once assigned, this value never changes.',
   'element_property': True,
   'minLength': 1,
   'maxLength': 64,
   'pattern': '^[A-Za-z0-9\\-.]+$',
   'type': 'string'},
  '_id': {'title': 'Extension field for ``id``.',
   'type': 'FHIR

In [10]:
polar_bundle_1 = "./polar/bundles/POLAR_Testdaten_UKB.json"

bundle_json = json.load(open(polar_bundle_1))
bundle = validate_bundle(bundle_json, drop_invalid_entries=True, show_invalid_entries=False)


In [21]:
print(blaze_api)
upload_bundle(bundle, fhir_api_url=blaze_api, token=token)




http://localhost:8080/fhir


{'id': 'C7A3SLLXT4HKDXHN',
 'type': 'transaction-response',
 'entry': [{'response': {'status': '201',
    'etag': 'W/"1"',
    'lastModified': '2021-09-25T06:09:40.069Z',
    'location': 'http://localhost:8080/fhir/Encounter/C7A3SLJEDTCPJXCR/_history/1'}},
  {'response': {'status': '201',
    'etag': 'W/"1"',
    'lastModified': '2021-09-25T06:09:40.069Z',
    'location': 'http://localhost:8080/fhir/Encounter/C7A3SLJEDTCPJXCS/_history/1'}},
  {'response': {'status': '201',
    'etag': 'W/"1"',
    'lastModified': '2021-09-25T06:09:40.069Z',
    'location': 'http://localhost:8080/fhir/Encounter/C7A3SLJEDTCPJXCT/_history/1'}},
  {'response': {'status': '201',
    'etag': 'W/"1"',
    'lastModified': '2021-09-25T06:09:40.069Z',
    'location': 'http://localhost:8080/fhir/Encounter/C7A3SLJEDTCPJXCU/_history/1'}},
  {'response': {'status': '201',
    'etag': 'W/"1"',
    'lastModified': '2021-09-25T06:09:40.069Z',
    'location': 'http://localhost:8080/fhir/Encounter/C7A3SLJEDTCPJXCV/_histo

In [14]:

query_df = query_server(resource="Encounter",
                        fhir_server_url=blaze_api,
                        token=token,
                        fhir_server_type="blaze",
                        out_format="raw")
query_df

ConnectionError: HTTPConnectionPool(host='localhost', port=8080): Max retries exceeded with url: /fhir/Encounter?_count=1000 (Caused by NewConnectionError('<urllib3.connection.HTTPConnection object at 0x000001EC3A8D6F40>: Failed to establish a new connection: [WinError 10061] No connection could be made because the target machine actively refused it'))

In [14]:

server = FhirServer(api_address=blaze_api, token=os.getenv("FHIR_TOKEN"))

In [16]:
response = server.raw_query("/Patient?", output_format="df")
response



HTTPError: 401 Client Error: Unauthorized for url: http://localhost:8080/fhir/Patient

## Cord FHIR resource generation based on csv



In [3]:
cord_test_data = pd.read_csv("./cord/A2-1.csv")


### Setup fhir server connections

In [4]:
load_dotenv(find_dotenv())

server_1_address = "http://193.196.20.24:9001/fhir"
server_2_address = "http://193.196.20.24:9002/fhir"
server_3_address = "http://193.196.20.24:9003/fhir"

client_id = os.getenv("CLIENT_ID")
client_secret = os.getenv("CLIENT_SECRET")
provider_url = os.getenv("OIDC_PROVIDER_URL")

server_1 = FhirServer(api_address=server_1_address, client_id=client_id, client_secret=client_secret,
                      oidc_provider_url=provider_url)
server_2 = FhirServer(api_address=server_2_address, client_id=client_id, client_secret=client_secret,
                      oidc_provider_url=provider_url)
server_3 = FhirServer(api_address=server_3_address, client_id=client_id, client_secret=client_secret,
                      oidc_provider_url=provider_url)

Requesting new token
Requesting new token
Requesting new token


### Create Condition resources based on csv file

In [5]:
code_system = "http://hl7.org/fhir/sid/icd-10"

def make_condition_resource(row):
    condition = Condition.construct()
    condition.clinicalStatus = CodeableConcept(
        coding=[Coding(
            **{
                "system": "http://hl7.org/fhir/condition-clinical",
                "code": "active",
                "display": "Active"
            }
        )]
    )

    condition.code = CodeableConcept(
        coding=[Coding(
            **{
                "system": code_system,
                "code": row["AngabeDiag1"],
                "display": row["TextDiagnose1"],
            }
        ).dict()]
    )
    return condition

cord_test_data["condition_resource"] = cord_test_data.apply(make_condition_resource, axis=1)

condition_resource_list = list(cord_test_data["condition_resource"])

### Distribute csv entries between stations

In [6]:
patients_per_station = int(len(cord_test_data) / 3)
patients_per_station


228

### Station 1
generate patients and assign a third of the csv resources to these patients

In [7]:
patient_generator = PatientGenerator(n=patients_per_station)
patient_generator.generate()
patient_bundle = patient_generator.make_bundle()

Generating 228 patients: 100%|██████████| 228/228 [00:00<00:00, 1266.68it/s]


In [8]:
patients_create_response_1 = server_1.add_bundle(patient_bundle)

### Assign a third of the resource to the generated patients

In [9]:
server_1_conditions = condition_resource_list[:patients_per_station]

references = patients_create_response_1.references
for i, condition in enumerate(server_1_conditions):
    # set the subject reference
    condition.subject = references[i]

In [10]:
response = server_1.add_all(server_1_conditions)


### Station 2

In [17]:
patient_generator = PatientGenerator(n=patients_per_station)
patient_generator.generate()
patient_bundle = patient_generator.make_bundle()

print(len(patient_bundle.entry))

Generating 228 patients: 100%|██████████| 228/228 [00:00<00:00, 1157.35it/s]

228


In [12]:
patients_create_response_2 = server_2.add_bundle(patient_bundle)

In [19]:
# check that all exist
print(len(patients_create_response_2.references))
for ref in patients_create_response_2.references:
    q = server_2.raw_query("/" + ref.reference)
    q.all()


684


HTTPError: 404 Client Error: Not Found for url: http://193.196.20.24:9002/fhir/Patient/C7FCA6G2ELOUOEGY

In [15]:
# second third of the resources defined in the csv
server_2_conditions = condition_resource_list[patients_per_station:2*patients_per_station]

references = patients_create_response_2.references
for i, condition in enumerate(server_2_conditions):
    # set the subject reference
    condition.subject = references[i]



In [14]:
response = server_2.add_all(server_2_conditions)

HTTPError: 409 Client Error: Conflict for url: http://193.196.20.24:9002/fhir

### Station 3

In [31]:
patient_generator = PatientGenerator(n=patients_per_station)
patient_generator.generate()
patient_bundle = patient_generator.make_bundle()

Generating 228 patients: 100%|██████████| 228/228 [00:00<00:00, 17534.59it/s]

228


In [28]:
patients_create_response_3 = server_3.add_bundle(patient_bundle)
print(len(patients_create_response_3.references))

1824


In [24]:
# check that all exist
for ref in patients_create_response_3.references:
    print(ref.reference)
    q = server_3.raw_query("/" + ref.reference)
    q.all()


Patient/C7FBP52CEH2XZPYF


HTTPError: 404 Client Error: Not Found for url: http://193.196.20.24:9003/fhir/Patient/C7FBP52CEH2XZPYF

In [ ]:
# second third of the resources defined in the csv
server_3_conditions = condition_resource_list[2*patients_per_station:]

references = patients_create_response_3.references
for i, condition in enumerate(server_3_conditions):
    # set the subject reference
    condition.subject = references[i]



In [ ]:
response = server_3.add_all(server_3_conditions)



In [45]:
load_dotenv(find_dotenv())
server_1_address = "http://193.196.20.24:9001/fhir"
client_id = os.getenv("CLIENT_ID")
client_secret = os.getenv("CLIENT_SECRET")
provider_url = os.getenv("OIDC_PROVIDER_URL")
server_1 = FhirServer(api_address=server_1_address, client_id=client_id, client_secret=client_secret,
                      oidc_provider_url=provider_url)

query = server_1.raw_query("/Patient/C7E67W3XOHARSS7L")
query.all()

Requesting new token
<class 'fhir.resources.patient.Patient'>


MissingSchema: Invalid URL '/Patient/C7E67W3XOHARSS7L': No schema supplied. Perhaps you meant http:///Patient/C7E67W3XOHARSS7L?